In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd

In [3]:
train = pd.read_csv("/content/drive/MyDrive/112_1/MachineLearning/train_new.csv", index_col=0)
test = pd.read_csv("/content/drive/MyDrive/112_1/MachineLearning/test_new.csv", index_col=0)

In [4]:
train0 = train[train["is_claim"] == 0].sample(1000, random_state=42)
train1 = train[train["is_claim"] == 1].sample(500, random_state=42)
train_new = pd.concat([train0, train1], ignore_index=False)
train_new = train_new.sample(frac=1, random_state=42, ignore_index=False)

In [5]:
x_train = train_new.drop(["is_claim"], axis="columns")
y_train = train_new["is_claim"]

In [6]:
def split(x, y, test_size=0.2):
    i = int((1 - test_size) * x.shape[0])
    o = np.random.permutation(x.shape[0])

    x_train, x_test = np.split(np.take(x,o,axis=0), [i])
    y_train, y_test = np.split(np.take(y,o), [i])
    return x_train, x_test, y_train, y_test

In [7]:
x_train, x_valid, y_train, y_valid = split(x_train, y_train, test_size=0.2)

In [8]:
# Utilize SHAP with your implemented algorithm to assess feature importance.
!pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 8.7 MB/s eta 0:00:00


In [9]:
import shap

In [10]:
def print_accuracy(f):
    print("Accuracy = {}%".format(100 * np.sum(f(x_valid) == y_valid) / y_valid.shape[0]))

In [11]:
from sklearn.tree import DecisionTreeClassifier

dtree = DecisionTreeClassifier(min_samples_split=2)
dtree.fit(x_train, y_train)
print_accuracy(dtree.predict)

# explain all the predictions in the test set
shap.initjs()
explainer = shap.KernelExplainer(dtree.predict_proba, x_train)
shap_values = explainer.shap_values(x_valid)
shap.force_plot(explainer.expected_value[0], shap_values[0], x_valid)

Accuracy = 59.0%


  0%|          | 0/300 [00:00<?, ?it/s]